In [1126]:
import pandas as pd
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import NMF
from sklearn.model_selection import GridSearchCV
import re

In [840]:
# load nltk's SnowballStemmer
from nltk.stem.snowball import SnowballStemmer
stemmer_eng = SnowballStemmer("english", ignore_stopwords=True)
stemmer_germ = SnowballStemmer("german", ignore_stopwords=True)

In [835]:
from spacy.lang.de import German
from spacy.lang.en import English
nlp_de = German()
nlp_en = English()

In [836]:
from bundeslander import Baden_Württemberg, Bayern, Berlin, Brandenburg, Bremen, Hamburg, Hessen, Mecklenburg_Vorpommern, Niedersachsen, Nordrhein_Westfalen, Rheinland_Pfalz, Saarland, Sachsen, Sachsen_Anhalt, Schleswig_Holstein, Thüringen, Ausland
All = Baden_Württemberg + Bayern + Berlin + Brandenburg + Bremen +Hamburg + Hessen + Mecklenburg_Vorpommern + Niedersachsen + Nordrhein_Westfalen + Rheinland_Pfalz + Saarland + Sachsen + Sachsen_Anhalt + Schleswig_Holstein + Thüringen + Ausland

In [801]:
cities = list(set([city.lower() for city in All]))

In [1135]:
df = pd.read_csv('../projectfinder.csv')
df.dropna(axis=0, inplace=True)
df.head()

,skill_summary,title,description
0,"tableau, visualisierung",Business Intelligence Analyst (m/w) - Tableau ...,Business Intelligence Analyst (m/w) - Tableau ...
1,", implementierung, java, ms sql, camel, apache...",Developer - Talend ESB oder Apache Camel (m/w)...,"Konzeption, Customizing sowie Softwareanpassun..."
2,", windows, clients, ios, mobile devices, mobil...",IT-Mitarbeiter (m/w) 1st / 2nd Level Support /...,"Als Mitglied eines kleinen, dynamischen Teams ..."
3,"dynamics ax 2009, dynamics crm 4.0, dynamics a...",Consultant für Microsoft Dynamics CRM (m/w),Über unseren Kunden: \n \n Unser Kunde ist e...
4,"hibernate, java, deutsch, oracle 11g, oracle 1...",Java Senior Developer - Backend (f/m),Über unseren Kunden: \n \n Unser Kunde ist e...


In [803]:
a = df[df.columns[0:]].apply(lambda x: ','.join(x.dropna()),axis=1).tolist()

In [1048]:
# load nltk's German and english stopwords'
import nltk
with open('german_stopwords_full.txt', 'r') as f:
    stopwords_germ = f.read().splitlines()

stopwords_eng = nltk.corpus.stopwords.words('english')
extra = ['deutscher','bewerben', 'bewerb', 'koln', 'direkt','herzlichen', 'melden', 'www', 'contactor', 'erfahrung', 'kenntnisse', 'kenntnis''contractor','https','de','impressum','munchen','frankfurt', 'suchen', 'kunden','bitte', 'inkl', 'bereich', 'einsatzort', 'dauer', 'umfeld', 'gmbh', 'pt', 'aufgaben', 'anforderungen', 'informationen', 'kunden', 'münchen', 'referenz', 'aufnehmen', 'profitieren', 'email', 'freuen', 'vorteile', 'ansprechpartner', 'beginn', 'qualifikation', 'referenznummer', 'unternehmen', 'dauer', 'passende', 'positionen', 'hays', 'kontakt', 'aufgaben']
extra1 = ['ort', 'customizing', 'freuen', 'fiori', 'bitte', 'basis', 'months', 'team', 'location', 'good', 'work', 'working', 'freitag', 'client', 'looking', 'knowledge', 'skills', 'experience', 'auftraggebers', 'jan', 'interessanten', 'selbstverständlich', 'bieten', 'freundlichen','understanding', 'service','firmensitz', 'fax', 'mobil', 'information', 'hrb', 'geschäftsführer','expertise', 'ready', 'honorarsatz', 'excellence', 'spannenden','abhängig', 'interessen', 'abwechslungsreiche', 'aussicht', 'aufgabe',]
extra2 = ['remote', 'interesse', 'laufzeit', 'freue', 'gerne', 'projekt', 'raum', 'auslastung','start', 'monate', 'erstellung','anzahl', 'personen', 'tagessatzes','völlig', 'bond', 'amoria', 'rahmenbedingungen', 'interessante', 'sicher', 'stellen', 'sektor', 'öffentlichen', 'experten','wachsen', 'frau', 'gestaltungsmöglichkeiten', 'arbeitsbereiches',  'grad', 'neueste', 'universität', 'gezielte', 'teamfähigkeit', 'engagierten', 'vorteil', 'mindestens', 'join', 'bewerbungsunterlagen', 'kommunikation', 'yrs','bitten']
extra3 = ['zeitraum', 'profil', 'senden', 'format', 'std','geehrte', 'damen', 'herren', 'verfügung', 'aktuell','woche','senior','möchten', 'gewünschten','new', 'product', 'strong', 'language','based', 'new', 'teams', 'contact', 'tasks','role', 'end', 'customer', 'required', 'michael', 'bailey', 'vermitteln', 'unterbreiten', 'pdf', 'intensiven','high', 'code', 'communication','holger', 'manager', 'anfragen', 'beratung', 'accounting', 'zukommen', 'lassen', 'munich','kompetenzen', 'verfügt', 'breites',  'individuellen', 'erbracht', 'standards', 'ag', 'isax', 'komponenten', 'abgabefrist', 'technischen', 'kompetent']
extra4 = ['königstein', 'percision', 'eu', 'id', 'verfügbarkeit', 'freuen', 'sofort', 'option', 'grüßen', 'profil', 'bitte', 'unseren', 'bewerbung', 'köln', 'mail', 'herr', 'zusendung','connect', 'alexander', 'nutzen','registrieren', 'rollout', 'angebot', 'erstellen', 'zusammenarbeit', 'rahmen', 'einfach','qualifikationsprofils', 'technologiedienstleistern', 'engineering', 'branchen', 'freiberuflicheinsatzort', 'technologiedienstleistern', 'technologies', 'engineer', 'engineering']
extra5 = ['deutsch','fachlichen', 'group','keil', 'montreal', 'associates', 'level', 'system', 'english', 'change', 'german', 'stundensatzvorstellung', 'wecken', 'führenden', 'weltweit', 'kontaktperson', 'kenntnissen', 'wort', 'schrift', 'spannende', 'herausgeben', 'weiterführende', 'spannendes', 'stundensätzen', 'preisvorgabe', 'bestmöglichen','engagierte', 'tellerrand', 'blicken', 'herausforderungen', 'wirtschaft', 'karten', 'sucht','honorierung', 'design', 'entsprechende', 'ausschließlich', 'zukünftigen', 'beachten', 'wien', 'sitz', 'arbeitszeiten', 'flexible', 'prozesse', 'schanzenstrasse', 'beratungsgesellschaft', 'mbh', 'technologien', 'homepage', 'direkte', 'falls', 'management']
extra6 = ['niederlassung', 'ferchau', 'ref', 'partner', 'bereichen', 'aussagekräftigen', 'angabe', 'projekte', 'einsatz', 'zusendung', 'arbeiten', 'betrieb', 'durchführung', 'erfahrungen', 'herr', 'fragen','passenden', 'bringen', 'mitarbeiter','zusätzliche','ausbildung', 'persönliche', 'fachhochschule', 'htl', 'aufgabengebiet', 'langjährigen','betroffenenrechte','registergericht', 'erfolgt', 'stets', 'spezialisten', 'erkennen', 'initiativ', 'persönlich', 'beantwortet', 'unterlagen', 'functional']
extra7 = ['fachwissen', 'führungskräfte', 'chance', 'fundiertes', 'angabe', 'level', 'top', 'verlängerungsoption', 'hierfür', 'englisch', 'umsetzung', 'unserem', 'deutsch', 'asap', 'kunde',  'deutsch', 'verfügbar','erstellen', 'zusammenarbeit','link', 'love', 'ulrich','spitzenunternehmen', 'solcom', 'entscheidet', 'zählt', 'global', 'agierender', 'innovation','template', 'ability','weiterentwicklung','controlling', 'sprache', 'schneller', 'empfehlen', 'entscheidung', 'telefonische', 'telefon']
extra8 = ['endkunden', 'cv', 'stundensatzes', 'aktuellen', 'hamburg', 'angabe', 'verlängerung', 'gesucht', 'vollzeit', 'unterstützung', 'standort', 'cv', 'using', 'please',  'germany',  'duration',  'german',  'years',  'english', 'angaben', 'alter', 'hinblick', 'bewerbungen', 'geschlecht', 'herzlichen', 'kontaktdaten', 'zustimmung', 'grüße', 'ariba', 'us', 'fokus', 'beste', 'services', 'stingel', 'voraus', 'geweckt','deutschkenntnisse','interessiert','noack','daten', 'swift', 'entwicklung', 'märz', 'berücksichtigen', 'recruitment','modern', 'qualifications', 'free',  'use', 'customers', 'bestätigen',  'einführung', 'details', 'branche','find', 'make', 'text']
extra9 = ['fragen', 'asap','recruiter', 'kandidaten', 'genannten', 'aktuellen','erfahrungen', 'kostenfrei', 'finden', 'asap', 'düsseldorf', 'telefonisch', 'standort', 'folgenden', 'stundensatzvorstellung', 'jederzeit', 'geforderten', 'rückfragen', 'vollzeit', 'deutsch', 'verlängerung', 'zusendung', 'per', 'hamburg', 'stehen',  'cv', 'erfahrungen', 'main', 'solution', 'job', 'ax', 'nav','projektdetails', 'anschluss', 'migration', 'vereinbart', 'kurzfristig', 'interviews',]
extra10 = ['komplexer', 'amtsgericht', 'geschäftsführung','datenschutz', 'automotive','umgang','wünschenswert', 'nice','within','ipsways', 'aix', 'arbeitsplatz', 'idealerweise', 'anfrage', 'möglichkeit', 'aktuelle', 'onsite', 'beschreibung', 'betreuung', 'nr','stundensatz', 'arbeitsende', 'arbeitsbeginn','zugang', 'reizvollen', 'größen', 'rekrutierung', 'angebote', 'erfahren', 'attraktive', 'kontakte','englischkenntnisse','science','mehrjährige', 'must', 'freelance', 'environment','logistik']
extra11 = [ 'interested','jahren','wordformat','tel','stellentyp','suche','projekten', 'office', 'jahre','tage','projektteams','kurzfristigen', 'dienstleistungen', 'rahmenvertragspartner', 'qualifizierte', 'motivierte', 'weinbergstraße', 'zuverlässig', 'spektrum', 'angebotsabgabe', 'grotz', 'fulltime', 'jobs', 'kolleginnen', 'zugesicherte','anlehnung','currently','anspruchsvolle', 'leistungsbereitschaft', 'projektbörse', 'archiviert', 'vakante', 'renommierten', 'ergebnisorientierung', 'word', 'profils', 'feedback', 'mitwirkung', 'projektbeschreibung', 'schicken', 'leistungsorientierte', 'aktiv', 'suche','fachausdrücke', 'diskriminierend', 'fundierte', 'qualifikationen','aktuelles', 'fähigkeiten','hause','verwenden', 'projekterfahrung','ust', 'kastenmeier', 'xxxx', 'co', 'board', 'deutschlandweit', 'projektteams', 'angebotsabgabe', 'grotz', 'fulltime', 'jobs','wordformat', 'kolleginnen', 'zugesicherte','tätigkeit', 'bergheimer', 'fundierte', 'tage', 'projektverlängerung', 'freiberuflichen', 'kreativität', 'erhebung', 'anspruchsvolle', 'leistungsbereitschaft', 'projektbörse', 'archiviert', 'vakante', 'renommierten', 'ergebnisorientierung', 'word', 'profils', 'feedback', 'mitwirkung', 'projektbeschreibung', 'schicken', 'leistungsorientierte','hinweg', 'rekrutierungserfahrung', 'kundenbeziehungen', 'endkundenprojekte', 'industriebranchen', 'bewerbungsprozess','abstimmung', 'alters', 'fachausdrücke', 'diskriminierend', 'fundierte', 'qualifikationen','rahmenverträge', 'anlagenbuchhaltung', 'branchenkompetenz', 'kommerzielle', 'entscheidende', 'acting', 'vacancy', 'suche', 'relation', 'unbefristet', 'erfolg', 'effizienz', 'daniel', 'kundenanfragen', 'freiberuflich', 'eventuell', 'besetzen', 'richten','able', 'written', 'month', 'including', 'provide', 'one', 'develop', 'area', 'time', 'quality', 'tools', 'like', 'industry', 'date', 'would', 'experienced','extension', 'international', 'excellent', 'systems', 'description', 'contract', 'projects', 'company', 'process', 'interested', 'fluent', 'currently', 'projektbeschreibung', 'optimale', 'ilhan', 'babac', 'schwerpunkt', 'finance', 'corporate', 'rufen', 'telefone', 'eventuell', 'innovativen', 'wünschen', 'realisierung', 'tel', 'zeit', 'interim', 'spezifikation', 'maximilian', 'projektort', 'phasen', 'projekten', 'tätigkeit', ]
ext = extra + extra1 + extra2 + extra3 + extra4 + extra5 + extra6 + extra7 + extra8 + extra9 + extra10 + extra11
len(ext)

652

In [1049]:
all_extras = list(set(extra + extra1 + extra2 + extra3 + extra4 + extra5 + extra6 + extra7 + extra8 + extra9 + extra10 + extra11))
stopwords = stopwords_germ + stopwords_eng + all_extras + cities
len(all_extras)

577

In [952]:
print(stemmer_germ.stem("erfahrung"))

erfahr


In [953]:
def tokenize_lower_alpha(text):
    tokens = nlp_de(text)
    data = list({token.lower_ for token in tokens if token.is_alpha})
    return data

In [954]:
tok = tokenize_lower_alpha('Working Jumping running 1234 kenntnisse Jumping softwarearchitektur softwarearchitektur kandidaten')
tok

['kandidaten',
 'softwarearchitektur',
 'jumping',
 'working',
 'running',
 'kenntnisse']

In [897]:
def lemmatse_stem_word(text):
    data = list({stemmer_germ.stem(token) for token in text if token not in stopwords})
    data = list({stemmer_eng.stem(token) for token in text if token not in stopwords})
    return data

In [898]:
lemmatse_stem_word(tok)

['jump', 'run', 'softwarearchitektur']

In [955]:
def remove_stopwords(data):
    data = list({token for token in data if token not in stopwords})
    return data

In [956]:
rem = remove_stopwords(tok)
rem

['jumping', 'running', 'softwarearchitektur']

In [957]:
def df_tolist(df):
    data = df[df.columns[0:]].apply(lambda x: ','.join(x.dropna()),axis=1).tolist()
    data = [re.sub('\s+', ' ', sent) for sent in data]
    return data

<>:3: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \s
<>:3: DeprecationWarning: invalid escape sequence \s
<ipython-input-957-d78d277f9180>:3: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [853]:
data = df_tolist(df)

In [1050]:
def cleaned_data(data):
    #data = [item.split() for item in data]
    data = [tokenize_lower_alpha(text) for text in data]
    data = [remove_stopwords(word) for word in data]
    #data = [lemmatse_stem_word(word) for word in data]
    #data = str(data).replace('[','').replace(']','').replace("'", "")
    return data

In [1051]:
prepared_data = cleaned_data(data)

In [1052]:
prepared_data = [' '.join(doc) for doc in prepared_data]

In [1053]:
prepared_data[:3]

['kundenprojekt kontinui analyst desktop visualisierung ffm business intelligence tableau',
 'softwarearchitekturen softwarearchitektur oracle webservices testautomatisierung camel ms talend java activemq esb datenmodelle sql softwaresystemen implementierung analyse developer datenbanken softwareanpassungen apache',
 'probleme priorisierung selbstständige dynamischen microsoft lösen bearbeitung pflege ios dokumentation telefon ausgeprägte referenzkunden clients sorgfältige windows entgegennahme muttersprache niveau zuordnung fehlerbehebung systemen mitglied support schichten gemeinsam ticket ms android einsatzbereitschaft gutes mobile freundliches übernahme problems lösungsansätze problemlösung montag kollegen devices analyse outlook servicezeiten']

In [937]:
def tokenize(text):
    lda_tokens = []
    tokens = nlp_de(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.is_alpha:
            #germ_lemma = token.lemma_
            #eng_stem = stemmer_eng.stem(token.lower_)
            #germ_stem= stemmer_germ.stem(token.lemma_)
            lower = re.sub("\'", "", token.lower_)
            if lower not in stopwords:
                lda_tokens.append(lower)
    return lda_tokens

In [708]:
def clean_and_stem(word):
    clean_word1 = [tokenize(a) for a in word]
    clean_word = str(clean_word1).replace('[','').replace(']','').replace("'", "")
    return clean_word

In [663]:
def get_optimal_data(df):
    #split features
    col = [col for col in df.columns]
    combined = df[col[0]] + ' '  + df[col[1]] + ' '  + df[col[2]]
    combined_tolist = [item.split() for item in combined]
    combined_cleaned = [clean_and_stem(word) for word in combined_tolist]
    return combined_cleaned

In [664]:
optimal_data = get_optimal_data(df)

In [666]:
optimal_data[:2]

['tableau, visualisierung, business, intelligence, analyst, , , tableau, desktop, , ffm, business, intelligence, analyst, , , tableau, desktop, für, einen, kundenprojekt, suchen, ich, ich, als, business, intelligence, analyst, , , tableau, desktop, der, kunde, haben, einen, sehr, hoch, kontinui, ',
 ', implementierung, java, ms, sql, camel, apache, camel, datenmodelle, analyse, engineering, entwicklung, oracle, datenbanken, talend, esb, ms, sql, apache, softwarearchitektur, activemq, testautomatisierung, webservices, developer, , talend, esb, oder, apache, camel, , , , , konzeption, customizing, sowie, softwareanpassungen, mit, talend, esb, implementierung, von, softwaresystemen, mit, java, analyse, sowie, design, von, softwarearchitekturen']

In [1054]:
#initialize vectorizer
tfidf = TfidfVectorizer(min_df=10, stop_words=stopwords, analyzer='word', ngram_range=(1,2))
cv = CountVectorizer(min_df=10, stop_words=stopwords, analyzer='word', ngram_range=(1,2))

In [1055]:
#fit vectorizer to df
cm = cv.fit_transform(prepared_data)
tf = tfidf.fit_transform(prepared_data)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['12', '2016', 'admannshagen', 'ahlefeld', 'ahrenshagen', 'ahrweiler', 'algesheim', 'alsenborn', 'annaberg', 'anspach', 'apenburg', 'apfelstädt', 'aspern', 'assenheim', 'audorf', 'auma', 'balbini', 'ballrechten', 'barchfeld', 'bargen', 'bargeshagen', 'bargfeld', 'barl', 'bartenshagen', 'baumbach', 'bayerfeld', 'behren', 'beierfeld', 'belgern', 'berggießhübel', 'berkholz', 'bernkastel', 'bernsbach', 'bertsdorf', 'berzdorf', 'besenbek', 'bickelheim', 'bieberau', 'bielatal', 'bienenmühle', 'birken', 'birkendorf', 'bistensee', 'bitterfeld', 'blankenfelde', 'bleyen', 'bobritzsch', 'bodenrode', 'bodman', 'bokholt', 'borgdorf', 'bornhofen', 'bornstein', 'brande', 'brehna', 'bretnig', 'brieskow', 'brotterode', 'bruchmühlbach', 'bruttig', 'burgalben', 'byhleguhre', 'byhlen', 'bäumenheim', 'böhl', '

# USING LDA

In [1056]:
#init LDA model
LDA = LatentDirichletAllocation(random_state=42, n_components=8)

In [1057]:
#fit model
LDA.fit(cm)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=8, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [1058]:
#extract features
features = cv.get_feature_names()

In [1059]:
len(features)

5605

In [1060]:
#get all topics probability distributions created by the LDA model
all_topic = LDA.components_
all_topic

array([[10.4030462 ,  0.125     ,  2.97128464, ...,  1.12453266,
         0.12502789, 15.12472751],
       [58.9389939 ,  0.125     , 22.82587811, ...,  0.125     ,
         2.24927404,  0.125     ],
       [10.92363151,  0.12501367,  0.12718678, ...,  3.76729387,
         0.12522913,  0.12505423],
       ...,
       [ 0.12506277,  0.12500001,  0.12502201, ...,  0.12501511,
         0.12574059,  0.125     ],
       [ 0.12504627,  0.12501541,  0.12501327, ...,  0.12504084,
        17.63299706,  0.125063  ],
       [ 0.1250552 ,  0.12500201,  0.12504739, ...,  5.48271615,
         5.23417127,  0.12500207]])

In [1061]:
#view top ten words in each topic currently 6
#enumurate creates a tuple of the index and the value
for i,topic in enumerate(all_topic):
    print(f'THE TOP 20 WORDS FOR GROUP #{i}')
    print([features[i] for i in topic.argsort()[-20:]])
    print()
   

THE TOP 20 WORDS FOR GROUP #0
['java', 'cloud', 'einverständnis', 'big', 'löschung', 'python', 'einsendeschluss', 'hadoop', 'erklären', 'erreichen', 'weitergabe', 'einsendeschluß', 'spark', 'sql', 'berater', 'profilvorgabe', 'registrierte', 'ausschreibung', 'geschäftspartner', 'data']

THE TOP 20 WORDS FOR GROUP #1
['testing', 'analysis', 'applications', 'agile', 'documentation', 'solutions', 'processes', 'software', 'consultant', 'application', 'developer', 'technical', 'sap', 'data', 'implementation', 'requirements', 'support', 'business', 'development', 'project']

THE TOP 20 WORDS FOR GROUP #2
['jira', 'git', 'analyse', 'integration', 'architektur', 'programmierung', 'contractor', 'konzeption', 'agilen', 'experts', 'java', 'datenschutzerklärung', 'javascript', 'consulting', 'dokumentation', 'software', 'scrum', 'implementierung', 'softwareentwicklung', 'entwickler']

THE TOP 20 WORDS FOR GROUP #3
['fachliche', 'kollegen', 'projektleiter', 'arbeitsweise', 'analyst', 'verständnis', '

In [1064]:
topics = LDA.transform(cm)

In [1065]:
topics.shape

(5360, 8)

In [1067]:
topics[0].round(2)

array([0.01, 0.01, 0.01, 0.85, 0.01, 0.01, 0.01, 0.08])

In [1068]:
topics

array([[0.01250658, 0.01251095, 0.01251152, ..., 0.01251007, 0.01250301,
        0.07903783],
       [0.00595961, 0.00595506, 0.31619811, ..., 0.06789481, 0.58611743,
        0.00595831],
       [0.0033816 , 0.04195836, 0.00338338, ..., 0.00337948, 0.07256467,
        0.79873426],
       ...,
       [0.00543952, 0.00545686, 0.00544069, ..., 0.00543609, 0.59297858,
        0.00543828],
       [0.00192414, 0.55664596, 0.09005156, ..., 0.00192637, 0.34368034,
        0.00192365],
       [0.34520488, 0.08033737, 0.00481179, ..., 0.15523955, 0.39996263,
        0.0048267 ]])

In [1097]:
topics[5875].round(2)

IndexError: index 5875 is out of bounds for axis 0 with size 5360

In [1072]:
# Create Document - Topic Matrix
#lda_output = best_lda_model.transform(cm)

# column names
topicnames = [i for i in range(8)]
names = []

for n, i in enumerate(topicnames):
    if i == 0:
        topicnames[n] = "Data_Engr/Big_Data"
    elif i == 1:
        topicnames[n] = "IT_Process_Mgr"
    elif i == 2:
        topicnames[n] = "IT_PM/Architect"
    if i == 3:
        topicnames[n] = "Business_Analyst/Consulting"
    if i == 4:
        topicnames[n] = "SAP/ERP"
    if i == 5:
        topicnames[n] = "SW_Dev/DevOps"
    if i == 6:
        topicnames[n] = "SW_Dev/Web"
    if i == 7:
        topicnames[n] = "IT_Admin/Ops"


# index names
docnames = ["Project" + str(i) for i in range(len(prepared_data))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(topics, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_bereich'] = dominant_topic
df_document_topic['dominant'] = dominant_topic
#df_document_topic['dominant_bereich'].replace()
#df['BrandName'].replace(['ABC', 'AB'], 'A')
#w['female'] = w['female'].map({'female': 1, 'male': 0})
df_document_topic['dominant_bereich'] = df_document_topic['dominant_bereich'].map({0: "Data_Engr/Big_Data", 1:"IT_Process_Mgr", 2: "IT_PM/Architect", 3: "Business_Analyst/Consulting", 4:"SAP/ERP", 5:"SW_Dev/DevOps", 6:"SW_Dev/Web", 7:"IT_Admin/Ops"})
# Styling
#def color_green(val):
    #color = 'green' if val > .1 else 'black'
    #return 'color: {col}'.format(col=color)

#def make_bold(val):
   # weight = 700 if val > .1 else 400
    #return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(15)
df_document_topics

,Data_Engr/Big_Data,IT_Process_Mgr,IT_PM/Architect,Business_Analyst/Consulting,SAP/ERP,SW_Dev/DevOps,SW_Dev/Web,IT_Admin/Ops,dominant_bereich,dominant
Project0,0.01,0.01,0.01,0.85,0.01,0.01,0.01,0.08,Business_Analyst/Consulting,3
Project1,0.01,0.01,0.32,0.01,0.01,0.07,0.59,0.01,SW_Dev/Web,6
Project2,0.00,0.04,0.00,0.07,0.00,0.00,0.07,0.80,IT_Admin/Ops,7
Project3,0.05,0.00,0.07,0.57,0.00,0.00,0.00,0.29,Business_Analyst/Consulting,3
Project4,0.00,0.00,0.00,0.00,0.00,0.00,0.98,0.00,SW_Dev/Web,6
Project5,0.00,0.00,0.24,0.36,0.00,0.00,0.00,0.39,IT_Admin/Ops,7
Project6,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.93,IT_Admin/Ops,7
Project7,0.00,0.05,0.62,0.16,0.00,0.00,0.17,0.00,IT_PM/Architect,2
Project8,0.00,0.00,0.00,0.00,0.00,0.00,0.45,0.52,IT_Admin/Ops,7
Project9,0.00,0.00,0.00,0.59,0.00,0.05,0.29,0.05,Business_Analyst/Consulting,3


In [1112]:
df['dominant_bereich'] = dominant_topic
df['dominant'] = dominant_topic
df['dominant_bereich'] = df['dominant_bereich'].map({0: "Data_Engr/Big_Data", 1:"IT_Process_Mgr", 2: "IT_PM/Architect", 3: "Business_Analyst/Consulting", 4:"SAP/ERP", 5:"SW_Dev/DevOps", 6:"SW_Dev/Web", 7:"IT_Admin/Ops"})
df.head(15)

,skill_summary,title,description,dominant_bereich,dominant
0,"tableau, visualisierung",Business Intelligence Analyst (m/w) - Tableau ...,Business Intelligence Analyst (m/w) - Tableau ...,Business_Analyst/Consulting,3
1,", implementierung, java, ms sql, camel, apache...",Developer - Talend ESB oder Apache Camel (m/w)...,"Konzeption, Customizing sowie Softwareanpassun...",SW_Dev/Web,6
2,", windows, clients, ios, mobile devices, mobil...",IT-Mitarbeiter (m/w) 1st / 2nd Level Support /...,"Als Mitglied eines kleinen, dynamischen Teams ...",IT_Admin/Ops,7
3,"dynamics ax 2009, dynamics crm 4.0, dynamics a...",Consultant für Microsoft Dynamics CRM (m/w),Über unseren Kunden: \n \n Unser Kunde ist e...,Business_Analyst/Consulting,3
4,"hibernate, java, deutsch, oracle 11g, oracle 1...",Java Senior Developer - Backend (f/m),Über unseren Kunden: \n \n Unser Kunde ist e...,SW_Dev/Web,6
5,"word 2010, vba, deutsch, javascript, hardware,...",DMS - CMS Consultant mit großer Prozeßerfahrun...,Über unseren Kunden: \n \n Unser Kunde ist T...,IT_Admin/Ops,7
6,"steuerungstechnik, industrie 4.0, siemens, vis...",Digitalisierung Galvanik\n ...,Digitalisierung in der Galvanik. \r\nKonzeptio...,IT_Admin/Ops,7
7,"app-entwicklung, deutsch, scrum, englisch, htm...",UX Designer für Web und App (m/w),Über unseren Kunden: \n \n Unser Kunde ist e...,IT_PM/Architect,2
8,"apache webserver, synapse, oracle datawarehous...",Linux Systemadministrator (m/w),Ihre Aufgaben: \n \n Mitarbeit in einem IT...,IT_Admin/Ops,7
9,"java, bpmn, scrum, agilität, sql, oo-objektori...",Requirements Engineer - Wertpapiere (m/w),Über unseren Kunden: \n \n Für unseren Kunde...,Business_Analyst/Consulting,3


In [1098]:
df.shape

(5360, 5)

In [1108]:
topics[5353].round(1)

array([0. , 0. , 0. , 0. , 0. , 0. , 0.9, 0. ])

In [1074]:
df_topic_distribution = df_document_topic['dominant'].value_counts().reset_index(name="Num Documents")
df_topic_distribution.columns = ['Topic Num', 'Num Documents']
df_topic_distribution

,Topic Num,Num Documents
0,6,1045
1,3,978
2,7,858
3,1,844
4,2,820
5,4,486
6,5,181
7,0,148


In [1076]:
df['dominant_bereich'].value_counts()

SW_Dev/Web                     1045
Business_Analyst/Consulting     978
IT_Admin/Ops                    858
IT_Process_Mgr                  844
IT_PM/Architect                 820
SAP/ERP                         486
SW_Dev/DevOps                   181
Data_Engr/Big_Data              148
Name: dominant_bereich, dtype: int64

In [1111]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(LDA, cm, cv, mds='tsne')
panel

/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]
/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.274284 -0.132680       1        1  19.799843
2     -0.061537  0.156550       2        1  16.336295
6     -0.036242  0.167447       3        1  15.971845
3     -0.153951 -0.056861       4        1  15.227773
7     -0.136568 -0.040411       5        1  15.101668
4     -0.161822 -0.208544       6        1   7.711168
0      0.189505 -0.056315       7        1   5.043620
5      0.086331  0.170814       8        1   4.807788, topic_info=     Category         Freq                 Term        Total  loglift  logprob
term                                                                          
4204  Default  1547.000000                  sap  1547.000000  30.0000  30.0000
2563  Default  1317.000000                 java  1317.000000  29.0000  29.0000
646   Default   682.000000              berater   682.000000  28.0000  28.0000
1596  Default  1271.000000           entwickler  1271.000000  27.0000  27.0000
5446  Default   536.000000              windows   536.000000  26.0000  26.0000
4580  Default   453.000000               spring   453.000000  25.0000  25.0000
3800  Default   521.000000              project   521.000000  24.0000  24.0000
2591  Default   666.000000           javascript   666.000000  23.0000  23.0000
1104  Default   520.000000                 data   520.000000  22.0000  22.0000
4486  Default   475.000000  softwareentwicklung   475.000000  21.0000  21.0000
78    Default   412.000000       administration   412.000000  20.0000  20.0000
0     Default   335.000000                 abap   335.000000  19.0000  19.0000
4755  Default   798.000000              support   798.000000  18.0000  18.0000
4297  Default   569.000000                scrum   569.000000  17.0000  17.0000
224   Default   374.000000              angular   374.000000  16.0000  16.0000
3186  Default   322.000000                   mm   322.000000  15.0000  15.0000
1331  Default   405.000000               docker   405.000000  14.0000  14.0000
2378  Default   272.000000       implementation   272.000000  13.0000  13.0000
307   Default   393.000000          application   393.000000  12.0000  12.0000
1268  Default   282.000000               devops   282.000000  11.0000  11.0000
1235  Default   194.000000                  dev   194.000000  10.0000  10.0000
3438  Default   189.000000                  ops   189.000000   9.0000   9.0000
4355  Default   528.000000               server   528.000000   8.0000   8.0000
1251  Default   534.000000          development   534.000000   7.0000   7.0000
836   Default   748.000000             business   748.000000   6.0000   6.0000
111   Default   438.000000               agilen   438.000000   5.0000   5.0000
552   Default   350.000000              backend   350.000000   4.0000   4.0000
795   Default   243.000000                 boot   243.000000   3.0000   3.0000
4113  Default   396.000000                 rest   396.000000   2.0000   2.0000
1038  Default   204.000000           contractor   204.000000   1.0000   1.0000
...       ...          ...                  ...          ...      ...      ...
5502   Topic8    27.441435                xhtml    31.190555   2.9069  -5.9355
5225   Topic8    40.628924      verschlüsselung    51.566033   2.7966  -5.5431
1237   Topic8    33.724985           dev docker    41.816314   2.8199  -5.7294
3438   Topic8   102.588820                  ops   189.841349   2.4195  -4.6169
1297   Topic8    50.672107             digitale    73.944791   2.6570  -5.3222
1235   Topic8   100.282708                  dev   194.674553   2.3716  -4.6396
5391   Topic8    52.122023            websphere    83.055063   2.5690  -5.2940
2270   Topic8    34.565744            herzliche    46.695643   2.7341  -5.7047
2149   Topic8    50.244781               google    95.469680   2.3930  -5.3307
1268   Topic8    91.622358               devops   282.540442   1.9088  -4.7299
492    Topic8    42.

# USING NMF

In [1113]:
nmf_model = NMF(n_components=8,random_state=42)

In [1127]:
tftr = TfidfTransformer(smooth_idf=False)

In [1128]:
xtf = tftr.fit_transform(cm)

In [1129]:
nmf_model.fit(xtf)

NMF(alpha=0.0, beta_loss='frobenius', init=None, l1_ratio=0.0, max_iter=200,
  n_components=8, random_state=42, shuffle=False, solver='cd', tol=0.0001,
  verbose=0)

In [1130]:
allNmf = nmf_model.components_

In [1131]:
topic_results = nmf_model.transform(cm)

In [1133]:
topic_results[1]

array([0.18887267, 0.        , 0.01966392, 0.07465717, 0.        ,
       0.19360932, 0.        , 0.        ])

In [1134]:
#view top ten words in each topic currently 6
#enumurate creates a tuple of the index and the value
for i,topic in enumerate(allNmf):
    print(f'THE TOP 20 WORDS FOR GROUP #{i}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-20:]])
    print()

THE TOP 20 WORDS FOR GROUP #0
['komplexen', 'steuerung', 'studium', 'applikationen', 'integration', 'planung', 'dienstleister', 'software', 'architektur', 'technischer', 'schnittstellen', 'datenbanken', 'programmierung', 'projektmanagement', 'technische', 'konzeption', 'softwareentwicklung', 'implementierung', 'dokumentation', 'analyse']

THE TOP 20 WORDS FOR GROUP #1
['projektstart', 'spätestens', 'entwickler', 'mai', 'oo', 'erp', 'ewm', 'april', 'sap support', 'abap sap', 'pp', 'hauptaufgabe', 'consultant', 'hana', 'sd', 'fi', 'mm', 'abap', 'berater', 'sap']

THE TOP 20 WORDS FOR GROUP #2
['architecture', 'testing', 'agile', 'software', 'consultant', 'analysis', 'applications', 'documentation', 'support', 'solutions', 'processes', 'developer', 'application', 'data', 'technical', 'implementation', 'requirements', 'business', 'development', 'project']

THE TOP 20 WORDS FOR GROUP #3
['citrix', 'monitoring', 'windows administrator', 'infrastruktur', 'cisco', 'itil', 'powershell', 'netzwe